In [57]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [59]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


# Read the CSV and Perform Basic Data Cleaning

In [60]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [62]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [63]:
from sklearn.preprocessing import LabelEncoder
credit_df = df.apply(LabelEncoder().fit_transform)
credit_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,1,0,2590,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,1,0,1889,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,1,0,2292,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,1,0,3010,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,1,0,1742,...,409,0,0,0,34824,11407,276,6250,0,0


# Split the Data into Training and Testing

In [64]:
# Create our features
X = credit_df.drop(['loan_status'], axis=1)

# Create our target
y = credit_df['loan_status']

In [65]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,572.296903,12.982577,4126.615168,1.812779,3294.087856,0.669994,0.805542,0.0,1962.376855,0.217722,...,367.820219,57.069314,0.125972,0.0,27710.412107,24184.046704,295.617696,19966.952715,0.0,0.0
std,375.779218,9.508701,2367.621789,0.941313,1574.576387,0.719105,0.714932,0.0,971.922012,0.717338,...,67.053040,55.878931,0.336732,0.0,17546.728857,14497.653994,257.292302,13661.709989,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,289.000000,5.000000,2153.000000,1.000000,1930.000000,0.000000,0.000000,0.0,1255.000000,0.000000,...,347.000000,0.000000,0.000000,0.0,12014.000000,11764.000000,116.000000,7895.000000,0.0,0.0
50%,516.000000,11.000000,3828.000000,1.000000,3365.000000,1.000000,1.000000,0.0,1842.000000,0.000000,...,409.000000,48.000000,0.000000,0.0,26735.000000,23056.000000,221.000000,18429.000000,0.0,0.0
75%,848.000000,20.000000,5967.000000,3.000000,4665.000000,1.000000,1.000000,0.0,2532.000000,0.000000,...,409.000000,101.000000,0.000000,0.0,42861.000000,36149.000000,393.000000,31243.000000,0.0,0.0
max,1397.000000,51.000000,8809.000000,3.000000,6089.000000,2.000000,2.000000,0.0,5695.000000,17.000000,...,409.000000,155.000000,4.000000,0.0,59628.000000,51997.000000,1715.000000,46898.000000,0.0,0.0


In [66]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Counter(y_train)

Counter({1: 51366, 0: 246})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [69]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

clf = BalancedRandomForestClassifier(n_estimators = 100, random_state=1)
clf.fit(X_train, y_train)


BalancedRandomForestClassifier(random_state=1)

In [70]:
# Calculated the balanced accuracy score
y_pred = clf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7334492423751262

In [71]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   63,    38],
       [ 2683, 14421]], dtype=int64)

In [72]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.62      0.84      0.04      0.73      0.51       101
          1       1.00      0.84      0.62      0.91      0.73      0.54     17104

avg / total       0.99      0.84      0.63      0.91      0.73      0.54     17205



In [73]:
# List the features sorted in descending order by feature importance

importances = clf.feature_importances_
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.07646711017201446, 'total_rec_prncp'),
 (0.0681731200022894, 'last_pymnt_amnt'),
 (0.05699252404641979, 'total_pymnt'),
 (0.05424778840861491, 'total_pymnt_inv'),
 (0.04735017423439608, 'total_rec_int'),
 (0.031245651401774252, 'int_rate'),
 (0.022619745828795626, 'issue_d'),
 (0.02244519942392171, 'mths_since_recent_inq'),
 (0.020409215004023765, 'installment'),
 (0.01808337089191728, 'tot_hi_cred_lim'),
 (0.017004195413593538, 'bc_util'),
 (0.01640559494238341, 'dti'),
 (0.016118934293676203, 'mths_since_rcnt_il'),
 (0.01581716497633785, 'il_util'),
 (0.015630310867867408, 'max_bal_bc'),
 (0.015494921031886601, 'out_prncp'),
 (0.015281199871082264, 'out_prncp_inv'),
 (0.01470961095035287, 'mo_sin_old_il_acct'),
 (0.014693851231416522, 'mo_sin_old_rev_tl_op'),
 (0.01425370693271944, 'total_bal_ex_mort'),
 (0.014094010852862895, 'total_bc_limit'),
 (0.013976327240119397, 'revol_bal'),
 (0.013793435235420952, 'avg_cur_bal'),
 (0.013416065136567646, 'tot_cur_bal'),
 (0.01339859137918

### Easy Ensemble AdaBoost Classifier

In [75]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [76]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.931601605553446

In [78]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [  985, 16119]], dtype=int64)

In [79]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.92      0.94      0.16      0.93      0.87       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

